In [1]:
import os
import shutil
import cv2
import random
import glob
from skimage.util import random_noise
import numpy as np

# image_index = 994 carti tata
# image_index = 547 ionut

In [52]:
# for paper to paste logo on
book = r"S:\books_to_images_ionut"
book_photos = os.listdir(book)
book_photos = [os.path.join(book, file) for file in book_photos]

In [53]:
dest_path = r"S:\logo detection flatten dataset v2\logos in the wild files"

In [ ]:
# for flattened directory
path = r"S:\logodet3k yolo"
files = os.listdir(path)
files = [os.path.join(path, file) for file in files]

In [54]:
# for 1 level nested directory
path = r"S:\logos in the wild samples yolo"
files = os.listdir(path)
files = [os.path.join(path, file) for file in files]

all_files = []
dir_files = []
for filename in glob.iglob(path + '**/**', recursive=True):
#     dir
    if len(filename.split("\\")) == 3:
        if dir_files != []:
            all_files.append(dir_files)
        dir_files = []
#     file
    elif len(filename.split("\\")) == 4:
        dir_files.append(filename)
        
all_files.append(dir_files)

files = sum(all_files, [])

In [18]:
def resize_image(image):
    h, w, _ = image.shape
    
    ratio = 1024/h
    
    new_h = int(h*ratio)
    new_w = int(w*ratio)
    
    image = cv2.resize(image, (new_w, new_h), interpolation = cv2.INTER_AREA)
    return image

In [19]:
def augment_pdf_to_mobile(image):
    # Add random noise to simulate inaccuracies
    noisy_image = random_noise(image, mode='gaussian', var=0.01)

    # Simulate shadows by darkening the entire image
    shadow_intensity = random.uniform(0.4, 0.9)  # Adjust the shadow intensity based on your requirements
    shadowed_image = noisy_image * shadow_intensity

    return shadowed_image

In [20]:
def apply_random_perspective(image):
    
    image = augment_pdf_to_mobile(image)
    
    height, width = image.shape[:2]

    # Define the source and destination points for perspective transformation
    src_points = np.float32([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]])

    # Calculate the maximum displacement limit
    max_displacement = int(min(width, height) * 0.1)

    # Generate random positive displacements for each corner of the image
    displacement_range = np.random.randint(0, max_displacement + 1, size=(4, 2))

    dst_points = np.zeros((4, 2), dtype=np.float32)
    
    dst_points[0][0], dst_points[0][1] = src_points[0][0] + displacement_range[0][0], src_points[0][1] + displacement_range[0][1]
    dst_points[1][0], dst_points[1][1] = src_points[1][0] - displacement_range[1][0], src_points[1][1] + displacement_range[1][1]
    dst_points[2][0], dst_points[2][1] = src_points[2][0] - displacement_range[2][0], src_points[2][1] - displacement_range[2][1]
    dst_points[3][0], dst_points[3][1] = src_points[3][0] + displacement_range[3][0], src_points[3][1] - displacement_range[3][1]
    
    # Generate random perspective transformation matrix
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Apply the perspective transformation to the image
    transformed_image = cv2.warpPerspective(image, perspective_matrix, (width, height))

    return transformed_image, perspective_matrix

In [55]:
image_index = 0
scale_percent = 70 # percent of original size

In [22]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [ ]:
# loop through each 
for i in range(0, len(files), 2):
    
#     image with logos
    img = cv2.imread(files[i])
    img_height, img_width, _ = img.shape
    
    with open(files[i+1], "r") as f:
        lines = f.readlines()
        
    bboxes = []
        
    for line in lines:
        line = line.split()
        x_center = float(line[1]) * img_width
        y_center = float(line[2]) * img_height
        bbox_width = float(line[3]) * img_width
        bbox_height = float(line[4]) * img_height
        x_min = int(x_center - bbox_width / 2)
        y_min = int(y_center - bbox_height / 2)
        x_max = int(x_center + bbox_width / 2)
        y_max = int(y_center + bbox_height / 2)
        bboxes.append([x_min, y_min, x_max, y_max])
        
        
    for box in bboxes:
        x_min, y_min, x_max, y_max = box[0], box[1], box[2], box[3]
        logo = img[y_min:y_max, x_min:x_max]
        
        while logo.shape >= img.shape:
            width = int(logo.shape[1] * scale_percent / 100)
            height = int(logo.shape[0] * scale_percent / 100)
            dim = (width, height)
            logo = cv2.resize(logo, dim, interpolation = cv2.INTER_AREA)
        
        logo_height, logo_width, _ = logo.shape
            
#         aleg o pagina buna din carte
        while True:
            book_page = cv2.imread(book_photos[image_index])
            book_page = resize_image(book_page)
            
            book_page_height, book_page_width, _ = book_page.shape
            image_index += 1
            
            cv2.namedWindow("book page", cv2.WINDOW_AUTOSIZE)
            # Set the position of the window
            cv2.moveWindow("book page", 0, 0)
            
            cv2.imshow("book page",book_page)
            k = cv2.waitKey(0)
            if k == 115: # save image -> s key 115
                cv2.destroyAllWindows()
                continue
            else:  # next image -> d key 100
                cv2.destroyAllWindows()
                break

        roi_logo = [0, 0, book_page_width-logo_width, book_page_height-logo_height]
        
        while True:
        
            x_logo_position = random.randint(roi_logo[0], roi_logo[2])
            y_logo_position = random.randint(roi_logo[1], roi_logo[3])
            
            xmin_logo_position = random.randint(roi_logo[0], roi_logo[2])
            ymin_logo_position = random.randint(roi_logo[1], roi_logo[3])
            xmax_logo_position = xmin_logo_position + logo_width
            ymax_logo_position = ymin_logo_position + logo_height
            
            book_page_copy = book_page.copy()

            book_page_copy[ymin_logo_position:ymax_logo_position, xmin_logo_position:xmax_logo_position] = logo
            
            cv2.namedWindow("book page", cv2.WINDOW_AUTOSIZE)
            # Set the position of the window
            cv2.moveWindow("book page", 0, 0)
            
            cv2.imshow("book page",book_page_copy)
            k = cv2.waitKey(0)
            if k == 115: # save image -> s key 115
                cv2.destroyAllWindows()
                continue
            else:  # next image -> d key 100
                cv2.destroyAllWindows()
                break
                
#         blur image
        if random.random() < 0.5:
            book_page_copy = augment_pdf_to_mobile(book_page_copy)
        else:
#         transpose and change logo coords
            book_page_copy, transformation_matrix = apply_random_perspective(book_page_copy)
            book_page_copy_h, book_page_copy_w, _ = book_page_copy.shape

            logo_coords = np.array([[xmin_logo_position, ymin_logo_position],
                                    [xmax_logo_position, ymax_logo_position]],
                                   dtype='float32')
            logo_coords = np.array([logo_coords])

            logo_coords = cv2.perspectiveTransform(logo_coords, transformation_matrix)
            logo_coords = (np.rint(logo_coords)).astype(int)

            if logo_coords[0][0][0]-1 <= 0:
                xmin_logo_position = 0
            else:
                xmin_logo_position = logo_coords[0][0][0]-1


            if logo_coords[0][0][1]-1 <= 0:
                ymin_logo_position = 0
            else:
                ymin_logo_position = logo_coords[0][0][1]-1


            if logo_coords[0][1][0]+1 >= book_page_copy_w:
                xmax_logo_position = book_page_copy_w
            else:
                xmax_logo_position = logo_coords[0][1][0]+1


            if logo_coords[0][1][1]+1 >= book_page_copy_h:
                ymax_logo_position = book_page_copy_h
            else:
                ymax_logo_position = logo_coords[0][1][1]+1
                
                
        cv2.imshow("book page",book_page_copy)
        k = cv2.waitKey(0)
        cv2.destroyAllWindows()

        
#         write gt and image to folder
        jpg_filename = book_photos[image_index].split("\\")[-1]
        txt_filename = jpg_filename[:-3] + "txt"
        
        jpg_filename = os.path.join(dest_path, jpg_filename)
        txt_filename = os.path.join(dest_path, txt_filename)
        
#         when shifting happens its 255
        cv2.imwrite(jpg_filename, 255*book_page_copy)
        
        yolo_bboxes = xml_to_yolo_bbox([xmin_logo_position, ymin_logo_position, xmax_logo_position, ymax_logo_position],
                                       book_page_width,
                                       book_page_height)
        
        with open(txt_filename, "w") as f:
            f.write(f"0 {yolo_bboxes[0]} {yolo_bboxes[1]} {yolo_bboxes[2]} {yolo_bboxes[3]}")

In [56]:
# loop through each 
for i in range(0, len(files), 2):
    
#     image with logos
    img = cv2.imread(files[i])
    img_height, img_width, _ = img.shape
    
    with open(files[i+1], "r") as f:
        lines = f.readlines()
        
    bboxes = []
        
    for line in lines:
        line = line.split()
        x_center = float(line[1]) * img_width
        y_center = float(line[2]) * img_height
        bbox_width = float(line[3]) * img_width
        bbox_height = float(line[4]) * img_height
        if bbox_width > 35 and bbox_height> 35:
            x_min = int(x_center - bbox_width / 2)
            y_min = int(y_center - bbox_height / 2)
            x_max = int(x_center + bbox_width / 2)
            y_max = int(y_center + bbox_height / 2)
            bboxes.append([x_min, y_min, x_max, y_max])
        
        
    for box in bboxes:
        x_min, y_min, x_max, y_max = box[0], box[1], box[2], box[3]
        logo = img[y_min:y_max, x_min:x_max]
        
        logo_height, logo_width, _ = logo.shape
            
        book_page = cv2.imread(book_photos[image_index])
        book_page = resize_image(book_page)
        book_page_height, book_page_width, _ = book_page.shape

#         print(logo.shape, book_page.shape)
        while logo_height >= book_page_height or logo_width >= book_page_width:
            print("logo too big, resizing...")
            logo_width = int(logo.shape[1] * scale_percent / 100)
            logo_height = int(logo.shape[0] * scale_percent / 100)
            dim = (logo_width, logo_height)
            logo = cv2.resize(logo, dim, interpolation = cv2.INTER_AREA)

        image_index += 1

        roi_logo = [0, 0, book_page_width-logo_width, book_page_height-logo_height]
        
        x_logo_position = random.randint(roi_logo[0], roi_logo[2])
        y_logo_position = random.randint(roi_logo[1], roi_logo[3])

        xmin_logo_position = random.randint(roi_logo[0], roi_logo[2])
        ymin_logo_position = random.randint(roi_logo[1], roi_logo[3])
        xmax_logo_position = xmin_logo_position + logo_width
        ymax_logo_position = ymin_logo_position + logo_height

        book_page_copy = book_page.copy()

        book_page_copy[ymin_logo_position:ymax_logo_position, xmin_logo_position:xmax_logo_position] = logo

                
#         blur image
        if random.random() < 0.5:
            book_page_copy = augment_pdf_to_mobile(book_page_copy)
        else:
#         transpose and change logo coords
            book_page_copy, transformation_matrix = apply_random_perspective(book_page_copy)
            book_page_copy_h, book_page_copy_w, _ = book_page_copy.shape

            logo_coords = np.array([[xmin_logo_position, ymin_logo_position],
                                    [xmax_logo_position, ymax_logo_position]],
                                   dtype='float32')
            logo_coords = np.array([logo_coords])

            logo_coords = cv2.perspectiveTransform(logo_coords, transformation_matrix)
            logo_coords = (np.rint(logo_coords)).astype(int)

            if logo_coords[0][0][0]-1 <= 0:
                xmin_logo_position = 0
            else:
                xmin_logo_position = logo_coords[0][0][0]-1


            if logo_coords[0][0][1]-1 <= 0:
                ymin_logo_position = 0
            else:
                ymin_logo_position = logo_coords[0][0][1]-1


            if logo_coords[0][1][0]+1 >= book_page_copy_w:
                xmax_logo_position = book_page_copy_w
            else:
                xmax_logo_position = logo_coords[0][1][0]+1


            if logo_coords[0][1][1]+1 >= book_page_copy_h:
                ymax_logo_position = book_page_copy_h
            else:
                ymax_logo_position = logo_coords[0][1][1]+1

#         write gt and image to folder
        jpg_filename = book_photos[image_index].split("\\")[-1]
        txt_filename = jpg_filename[:-3] + "txt"
        
        jpg_filename = os.path.join(dest_path, jpg_filename)
        txt_filename = os.path.join(dest_path, txt_filename)
        
#         when shifting happens its 255
        cv2.imwrite(jpg_filename, 255*book_page_copy)
        
        yolo_bboxes = xml_to_yolo_bbox([xmin_logo_position, ymin_logo_position, xmax_logo_position, ymax_logo_position],
                                       book_page_width,
                                       book_page_height)
        
        with open(txt_filename, "w") as f:
            f.write(f"0 {yolo_bboxes[0]} {yolo_bboxes[1]} {yolo_bboxes[2]} {yolo_bboxes[3]}")

here
here
here
here
here
here
here
here
here
here
here
here
here
here
